<a href="https://colab.research.google.com/github/eliasbaumann/bnn-for-curiosity/blob/master/BNN_Curiosity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np

import threading
import queue
import matplotlib.pyplot as plt

# import mlagents
# import obstacle_tower_env 

In [0]:
### starting out by writing ppo
## good code example here: https://github.com/MorvanZhou/Reinforcement-learning-with-tensorflow/blob/master/contents/12_Proximal_Policy_Optimization/discrete_DPPO.py
## because its short and doesnt do anything extra
## Next step would be to implement curiosity. But this should be the minimal baseline because thats already implemented in the Unity MLagents toolkit...<

PATH = 'C:/Users/Elex/Downloads/obstacle-tower-challenge/ObstacleTower/obstacletower'

OBS_DIM = 84*84*3
ACTION_DIM = 54 # ALSO QUESTIONABLE IF THATS A GOOD IDEA -> LOOK AT WORK HOW TO DEAL WITH LARGE ACTION SPACE...
CRITIC_LR = 0.0001
ACTOR_LR = 0.0002
EPSILON = .2
GAMMA = .9

EPISODE_MAX = 1000
MIN_BATCH_SIZE = 64
UPDATE_STEP=10 #?????

NUMBER_OF_WORKERS = 4

class PPO(object):
  def _actor_network(self,name,trainable):
    with tf.variable_scope(name,reuse=tf.AUTO_REUSE):
      act = tf.layers.dense(self.inp,200,tf.nn.relu,trainable=trainable)
      
      
      # mu = 2* tf.layers.dense(act,ACTION_DIM,tf.nn.tanh,trainable=trainable)
      action_probs = tf.layers.dense(act,ACTION_DIM,tf.nn.softmax,trainable=trainable)
      # norm_dist = tf.distributions.Normal(loc=mu,scale=sigma)
    params = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,scope=name)
    return action_probs,params
  
  def __init__(self):
    self.sess = tf.Session()
    
    self.inp = tf.placeholder(tf.float32,[None,OBS_DIM],name='state')
    
    # ppo is based on actor critic 
    
    # Critic
    crit = tf.layers.dense(self.inp,100,tf.nn.relu)
    self.value = tf.layers.dense(crit,1)
    self.dc_reward = tf.placeholder(tf.float32,[None,1],name='discounted_r')
    self.advantage = self.dc_reward - self.value
    self.critic_loss = tf.reduce_mean(tf.square(self.advantage))
    self.critic_train_opt = tf.train.AdamOptimizer(CRITIC_LR).minimize(self.critic_loss)
    
    
    # Actor 
    self.policy, theta_p = self._actor_network('policy',trainable=True)
    old_policy, old_theta_p = self._actor_network('old_policy',trainable=False)
    
    self.update_old_policy_op =[oldp.assign(p) for p,oldp in zip(theta_p,old_theta_p)]
    
    self.action = tf.placeholder(tf.int32, [None,],name='action')
    self.full_adv = tf.placeholder(tf.float32, [None,1],name='advantage')
    
    a_indices = tf.stack([tf.range(tf.shape(self.action)[0], dtype=tf.int32), self.action], axis=1)
    pi_prob = tf.gather_nd(params=self.policy, indices=a_indices)   
    oldpi_prob = tf.gather_nd(params=old_policy, indices=a_indices) 
    ratio = pi_prob/oldpi_prob
    surr = ratio * self.full_adv
    
    
    self.actor_loss = -tf.reduce_mean(tf.minimum(surr,tf.clip_by_value(ratio,1.-EPSILON,1.+EPSILON)*self.full_adv))
    
    self.actor_train_opt = tf.train.AdamOptimizer(ACTOR_LR).minimize(self.actor_loss)
    
    self.sess.run(tf.global_variables_initializer())
    
    
  def get_action(self,state):
    action_probs = self.sess.run(self.policy, {self.inp:state})
    action = np.random.choice(range(action_probs.shape[1]),p=action_probs.ravel())
    return action

  def get_value(self,state):
    return self.sess.run(self.value,{self.inp:state})[0,0] #TODO same as with get_action, need to figure out what comes out of here...

  def update(self):
    global GLOBAL_UPDATE_COUNTER
    while not COORD.should_stop():
      if GLOBAL_EPISODE < EPISODE_MAX:
        UPDATE_EVENT.wait()
        self.sess.run(self.update_old_policy_op)
        data = [QUEUE.get() for _ in range(QUEUE.qsize())]
        data = np.vstack(data)
        state,action,reward = data[:, :OBS_DIM], data[:, OBS_DIM: OBS_DIM + 1].ravel(), data[:, -1:]
        advantage = self.sess.run(self.advantage, {self.inp: state,self.dc_reward: reward})
        # update actor and critic in a update loop
        [self.sess.run(self.actor_train_opt, {self.inp: state, self.action: action, self.full_adv: advantage}) for _ in range(UPDATE_STEP)]
        [self.sess.run(self.critic_train_opt, {self.inp: state, self.dc_reward: reward}) for _ in range(UPDATE_STEP)]
        UPDATE_EVENT.clear()        # updating finished
        GLOBAL_UPDATE_COUNTER = 0   # reset counter
        ROLLING_EVENT.set()         # set roll-out available

In [0]:
# Worker for multiple agents at the same time

class Worker(object):
  def __init__(self,wid):
    self.wid = wid
    self.env = obstacle_tower_env.ObstacleTowerEnv(PATH,retro=True,worker_id=wid)
    self.ppo = GLOBAL_PPO
  
  def vectorize_state(self,state):
    return None
  
  def work(self):
    global GLOBAL_EPISODE,GLOBAL_RUNNING_REWARD,GLOBAL_UPDATE_COUNTER
    while not COORD.should_stop():
      state = self.env.reset()
      state = np.expand_dims(state.flatten(), axis=0)
      episode_reward = 0
      done = False
      buffer_state,buffer_action,buffer_reward = [],[],[]
      while not done: 
        
        if not ROLLING_EVENT.is_set():
          ROLLING_EVENT.wait()
          buffer_state,buffer_action,buffer_reward = [],[],[]
        
        # Add multiple runs until batch size
        action = self.ppo.get_action(state)
        state_,reward,done,_ = self.env.step(action)
        state_ = np.expand_dims(state_.flatten(),axis=0)
        
        ### INSERT CURIOSITY HERE:
        # reward += new_reward
        
        buffer_state.append(state)
        buffer_action.append(action)
        buffer_reward.append(reward) #he normalized this, probably smart to do when we add curiosity as well...
        state = state_
        episode_reward += reward
        GLOBAL_UPDATE_COUNTER += 1
        
        # If enough state,action,reward triples are collected:
        if GLOBAL_UPDATE_COUNTER >= MIN_BATCH_SIZE or done:
          state_value = self.ppo.get_value(state_)
          discounted_reward = []
          for r in buffer_reward: #[::-1] he adds that, but need to check if needed
            state_value = r + GAMMA * state_value
            discounted_reward.append(state_value)
          discounted_reward.reverse()
          
          bs, ba, br = np.vstack(buffer_state),np.vstack(buffer_action),np.vstack(discounted_reward) #[:,np.newaxis] ??
          buffer_state,buffer_action,buffer_reward = [],[],[]
          QUEUE.put(np.hstack((bs,ba,br)))
          
          ROLLING_EVENT.clear()
          UPDATE_EVENT.set()
          
          if GLOBAL_EPISODE >= EPISODE_MAX:
            COORD.request_stop()
            break
      
      
      if len(GLOBAL_RUNNING_REWARD) == 0: 
        GLOBAL_RUNNING_REWARD.append(episode_reward)
      else:
        GLOBAL_RUNNING_REWARD.append(GLOBAL_RUNNING_REWARD[-1]*0.9+episode_reward*0.1)
      GLOBAL_EPISODE += 1
      print('{0:.1f}%'.format(GLOBAL_EPISODE/EPISODE_MAX*100), '|W%i' % self.wid,  '|Ep_r: %.2f' % episode_reward,)


In [0]:
### MAIN, in the actual implementation uncomment the next line and re-indent everything
# if __name__=='__main__':
tf.logging.set_verbosity(tf.logging.INFO)

GLOBAL_PPO = PPO()
UPDATE_EVENT, ROLLING_EVENT = threading.Event(), threading.Event()
UPDATE_EVENT.clear()
ROLLING_EVENT.set()
workers = [Worker(wid=i) for i in range(NUMBER_OF_WORKERS)]

GLOBAL_UPDATE_COUNTER = 0
GLOBAL_EPISODE = 0

GLOBAL_RUNNING_REWARD = []

COORD = tf.train.Coordinator()
QUEUE = queue.Queue()
threads = []
for worker in workers:
  thread = threading.Thread(target=worker.work,args=())
  thread.start()
  threads.append(thread)
ppo_update_thread = threading.Thread(target=GLOBAL_PPO.update)
ppo_update_thread.start()
threads.append(ppo_update_thread)
COORD.join(threads)

In [0]:
# TODO, calculate additional reward from curiosity 
STATE_SHAPE = [None,84,84,3]
STATE_LATENT_SHAPE = 64


class Curiosity(object):
  def __init__(self,sess,s_t,s_t1,a_t):
    self.sess = sess
    self.s_t = s_t
    self.s_t1 = s_t1
    self.a_t = a_t
    
    
    self.inp_st = tf.placeholder(tf.float32,STATE_SHAPE,name='S_t_input')
    # self.inp_st1 = tf.placeholder(tf.float32,STATE_SHAPE,name='S_t+1_input')
    self.inp_at = tf.placeholder(tf.float32,[None,],name = 'A_t_input')
    
  
  def cnn_feature_extractor(self):
    cn1 = tf.layers.conv2d()
  
  def forward_model(self):
    l1 = tf.layers.dense(tf.concat(),200,tf.nn.relu,trainable=trainable)
    
  def reverse_model(self):
    
  
  def get_reward(self):
    
  